In [ ]:
#Understanding necessary Astropy tools
from astropy import units as u 
from astropy import constants as cst 
from astropy.coordinates import SkyCoord
from astropy.table import QTable, Table

import numpy as np 

#Example of use: Units and Constants 
g_force  = (cst.G * cst.M_sun * cst.M_earth) / (u.au)**2
print(f'Gravitational force beetwen Sun and Earth: {g_force.to(u.N)}')

#Example of use: Coordinates 
position = SkyCoord(45.5*u.hourangle, 23.9*u.deg)
print(f'Normalized coordinates in degrees: {position}')
print(f'Normalized coordinates in hourangles: {position.ra.hms}')

#Example of use: QTable 
planets = ['Venus', 'Earth', 'Mars']
order = np.array([2,3,4])
my_rate = [8.8,10.0,4.5]
masses = [cst.M_earth*0.815, cst.M_earth, cst.M_earth*0.1075] #unit=kg
rock_planets_table = QTable([planets, order, masses, my_rate],
                             names=('Planets', 'Order', 'Masses', 'My_Rate'),
                               meta={'name':'Test Table'})

print(rock_planets_table)

Gravitational force beetwen Sun and Earth: 3.541545424043141e+22 N
Normalized coordinates in degrees: <SkyCoord (ICRS): (ra, dec) in deg
    (322.5, 23.9)>
Normalized coordinates in hourangles: hms_tuple(h=np.float64(21.0), m=np.float64(29.0), s=np.float64(59.99999999998721))
Planets Order         Masses        My_Rate
                        kg                 
------- ----- --------------------- -------
  Venus     2 4.867316812249974e+24     8.8
  Earth     3 5.972167867791379e+24    10.0
   Mars     4 6.420080457875733e+23     4.5
